# helpers

> This module contains all the helper functions for this library.

In [ ]:
#| default_exp helpers

In [ ]:
#| export
import code_tokenizers

from fastcore.script import *
from git import Git, Repo
from pathlib import Path
from tree_sitter import Language

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
from datasets import load_dataset

ds = load_dataset("bigcode/the-stack-smol", data_dir="data/python", split="train")

In [ ]:
from tqdm.auto import tqdm

repo_files = {}
for repo_name in tqdm(set(ds["repository_name"]), desc="Processing repos"):
    rows_w_repo = ds.filter(lambda example: example["repository_name"] == repo_name)

    if len(rows_w_repo) > 1:
        repo_files[repo_name] = [row["content"] for row in rows_w_repo]
        if len(repo_files) > 10:
            break

In [ ]:
repo_files.keys()

In [ ]:
#| export
def get_query(language, program_lang):
    "Get a query based on the language"
    if program_lang == "python":
        return language.query("""
            (function_definition
                name: (identifier) @func.name)

            (class_definition
                name: (identifier) @class.name)
            """
        )

In [ ]:
#| export
def get_definitions(file_contents, tokenizer):
    """
    Get all definitions from a given list of files
    """
    project_content = "\n\n".join(file_contents)
    tree = tokenizer.parser.parse(project_content.encode())
    root_node = tree.root_node
    query = get_query(tokenizer.language, tokenizer.program_lang)
    captures = query.captures(root_node)
    definitions = {node.text.decode() for node, _ in captures}
    return definitions

In [ ]:
from code_tokenizers.core import CodeTokenizer

py_tokenizer = CodeTokenizer.from_pretrained("gpt2", "python")

# definitions = get_definitions(repo_files["articuly/alipay-sdk-python-all"], py_tokenizer)

In [ ]:
# definitions

In [ ]:

# PY_LANGUAGE = Language('/workspace/main/scripts/build/my-languages.so', 'python')
# QUERY = PY_LANGUAGE.query("""
# (function_definition
# 	name: (identifier) @func.name)

# (class_definition
# 	name: (identifier) @class.name)
# """
# )

In [ ]:
# # Grab the node types from the tree-sitter language
# language = Language(f"{code_tokenizers.__path__[0]}/grammars/tree-sitter-languages.so", lang)
# node_path = f"{code_tokenizers.__path__[0]}/grammars/tree-sitter-{lang}/src/node-types.json"
# with open(node_path) as f:
#     node_types = json.load(f)
# node_types = unroll_node_types(node_types)
# if lang == "python":
#     node_types.append("as_pattern_target")

# # Create a parser for the language
# parser = Parser()
# parser.set_language(language)

In [1]:
#| hide
import nbdev; nbdev.nbdev_export()

ModuleNotFoundError: No module named 'nbdev'